In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('testData.csv')

In [3]:
df

,disclosureYear,disclosureDate,transactionDate,owner,ticker,assetDescription,type,amount,representative,district,capitalGainsOver200USD,option_symbol
0,2023,6/15/2023,5/20/2023,Spouse,AAPL,Apple Inc. Stock,Purchase,"$100,001 - $250,000",Nancy Pelosi,CA-12,Yes,NaN
1,2023,5/12/2023,4/10/2023,Self,GOOGL,Alphabet Inc. Stock,Sale (Full),"$50,001 - $100,000",Nancy Pelosi,CA-12,No,NaN
2,2023,12/29/2023,12/6/2022,Dependent,AMZN,AMZN Stock,Purchase,"$100,001 - $250,000",Nancy Pelosi,NY-14,Yes,NaN
3,2023,2/13/2022,4/14/2022,Self,AMZN,AMZN Stock,Purchase,"$100,001 - $250,000",Nancy Pelosi,FL-9,No,NaN
4,2023,6/23/2023,3/11/2022,Joint,MSFT,MSFT Stock,Purchase,"$1,001 - $15,000",Nancy Pelosi,CA-12,Yes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
97,2023,3/15/2023,7/26/2023,Dependent,AAPL,AAPL Stock,Sale (Full),"$1,001 - $15,000",Nancy Pelosi,FL-9,Yes,NaN
98,2022,6/8/2022,5/3/2023,Joint,TSLA,TSLA Stock,Sale (Full),"$50,001 - $100,000",Nancy Pelosi,CA-12,No,NaN
99,2023,7/24/2022,9/16/2023,Self,GOOGL,GOOGL Stock,Sale (Partial),"$1,001 - $15,000",Nancy Pelosi,TX-7,No,NaN
100,2023,12/25/2022,6/25/2022,Spouse,AAPL,AAPL Stock,Sale (Full),"$100,001 - $250,000",Nancy Pelosi,NY-14,Yes,NaN


In [66]:
df.shape ##### specified the row & columns

(102, 12)

# Question 1

Write a Python function to calculate the trade performance of a given set of trades executed by a financial firm. The function should take a pandas DataFrame as input with the following columns:

Answer :-
           To calculate the trade performance of a given set of trades executed by a financial firm, I implemented a function that computes ten financial metrics. These metrics will help in assessing the effectiveness and risks of the trading strategy. Below is the detailed implementation of the function with comments explaining each step and the choice of metrics.
           
                                   Explanation of Metrics and Calculations:
          
* Total Trades: Counts the total number of trades in the dataset.

* Buy Count and Sell Count: Counts the number of trades categorized as 'Purchase' and 'Sale' respectively.

* Total Capital Gains: Sums up the capital gains from all trades.

* Total Portfolio Return: Assumes total portfolio return is equal to total capital gains.

* Average Holding Period: Calculates the average holding period in days.

* Long Strategy Return: Sums up capital gains from trades categorized as 'Purchase'.

* Short Strategy Return: Sums up capital gains from trades categorized as 'Sale'.

* Win-Loss Ratio: Ratio of Buy Count to Sell Count, to gauge the balance between buying and selling.

* Sharpe Ratio: Placeholder value for now, typically requires risk-free rate and benchmark returns for accurate calculation.

* Max Drawdown: Calculates the maximum drawdown, which is the maximum loss from a peak to a trough of a portfolio.

                                      Handling Long-Short Strategies:
                                      
* Long Strategy Return: Assumes profits from selling higher than buying.

* Short Strategy Return: Differentiates based on trade types ('Short Borrowed' or 'Short Sale'), calculating returns accordingly.

                                          Handling Cases

* This function handles data cleaning (e.g., converting dates, numeric values), computes various trade performance metrics, and handles different trade types for both long and short strategies. Adjustments can be made to fit specific requirements or to include additional metrics as needed.

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime

# Example function to get ticker price (mock implementation)
def getTickerPrice(ticker: str, date: datetime) -> float:
    # Replace with actual implementation to fetch price for a given ticker and date
    return np.random.uniform(1, 100)

def calculate_trade_performance(trades_df):
    # Convert date columns to datetime format if needed
    trades_df['transactionDate'] = pd.to_datetime(trades_df['transactionDate'], errors='coerce')
    trades_df['disclosureDate'] = pd.to_datetime(trades_df['disclosureDate'], errors='coerce')

    # Convert amount and capitalGainsOver200USD to numeric
    trades_df['amount'] = pd.to_numeric(trades_df['amount'], errors='coerce')
    trades_df['capitalGainsOver200USD'] = pd.to_numeric(trades_df['capitalGainsOver200USD'], errors='coerce')

    # Initialize metrics dictionary
    metrics = {}

    if trades_df.empty:
        return metrics
    
    # 1. Total number of trades
    metrics['Total Trades'] = len(trades_df)

    # 2. Total buy and sell counts
    metrics['Buy Count'] = len(trades_df[trades_df['type'].str.contains('Purchase', case=False)])
    metrics['Sell Count'] = len(trades_df[trades_df['type'].str.contains('Sale', case=False)])

    # 3. Total capital gains
    capital_gains = trades_df['capitalGainsOver200USD'].sum()
    metrics['Total Capital Gains'] = capital_gains

    # 4. Total portfolio return (assuming gains are profits from selling)
    total_portfolio_return = capital_gains
    metrics['Total Portfolio Return'] = total_portfolio_return

    # 5. Average holding period
    trades_df['Holding Period'] = (trades_df['disclosureDate'] - trades_df['transactionDate']).dt.days
    metrics['Average Holding Period'] = trades_df['Holding Period'].mean()

    # 6. Long and short positions
    long_trades = trades_df[trades_df['type'].str.contains('Purchase', case=False)]
    short_trades = trades_df[trades_df['type'].str.contains('Sale', case=False)]

    # 7. Long strategy return (assuming sold higher than bought)
    long_returns = long_trades['capitalGainsOver200USD'].sum()
    metrics['Long Strategy Return'] = long_returns

    # 8. Short strategy return (handle short position types)
    short_returns = short_trades['capitalGainsOver200USD'].sum()
    metrics['Short Strategy Return'] = short_returns

    # Handle short positions based on type (add comments for clarity)
    if 'Short Borrowed' in trades_df['type'].unique():
        # Calculate return for borrowing and selling
        short_returns = (short_trades['amount'] + short_trades['capitalGainsOver200USD']).sum()
    elif 'Short Sale' in trades_df['type'].unique():
        # Calculate return for selling short first (assuming covering later at higher price)
        short_returns = (short_trades['capitalGainsOver200USD'] - short_trades['amount']).sum()

    metrics['Short Strategy Return'] = short_returns

    # Calculate Win-Loss Ratio
    if metrics['Sell Count'] > 0:
        metrics['Win-Loss Ratio'] = metrics['Buy Count'] / metrics['Sell Count']
    else:
        metrics['Win-Loss Ratio'] = 0  # Handle division by zero case

    # Placeholder value for Sharpe Ratio (requires risk-free rate and benchmark returns)
    metrics['Sharpe Ratio'] = 0.5  # Placeholder value

    # Calculate Max Drawdown
    sale_amounts = trades_df.loc[trades_df['type'].str.contains('Sale', case=False), 'amount'].values
    purchase_amounts = trades_df.loc[trades_df['type'].str.contains('Purchase', case=False), 'amount'].values

    cumulative_return_sale = np.cumsum(sale_amounts)
    cumulative_return_purchase = np.cumsum(purchase_amounts)

    if len(cumulative_return_sale) > len(cumulative_return_purchase):
        cumulative_return_sale = cumulative_return_sale[:len(cumulative_return_purchase)]
    elif len(cumulative_return_purchase) > len(cumulative_return_sale):
        cumulative_return_purchase = cumulative_return_purchase[:len(cumulative_return_sale)]

    max_drawdown = np.max(np.maximum.accumulate(cumulative_return_purchase) - cumulative_return_sale) / np.max(np.maximum.accumulate(cumulative_return_purchase))
    metrics['Max Drawdown'] = max_drawdown

    return metrics

# Load data from CSV file
trades_df = pd.read_csv('testData.csv')

# Calculate trade performance metrics
performance_metrics = calculate_trade_performance(trades_df)

# Print all calculated metrics 
for key, value in performance_metrics.items():
    if key not in ['Total Portfolio Return']:
        print(f"{key}: {value}")


Total Trades: 102
Buy Count: 40
Sell Count: 62
Total Capital Gains: 0.0
Average Holding Period: 39.53921568627451
Long Strategy Return: 0.0
Short Strategy Return: 0.0
Win-Loss Ratio: 0.6451612903225806
Sharpe Ratio: 0.5
Max Drawdown: nan


# Question 2: Analyzing Nancy Pelosi's Trading Performance

    Step 1: Load and Preprocess Data
First, I ensure the testData.csv file is loaded into a pandas DataFrame and preprocess it to match the required format for calculate_trade_performance function:

    Step 2: Ensure Column Names and Types
# Rename columns to match the expected format
trades_df = trades_df.rename(columns={
    'Date': 'transactionDate',
    'Symbol': 'ticker',
    'Side': 'type',
    'Size': 'amount',
    'Price': 'capitalGainsOver200USD',
    'DisclosureDate': 'disclosureDate'
})

     Step 3: Calculate Trade Performance Metrics
Now, applied the calculate_trade_performance function to compute the trade performance metrics:
#Imported the function calculate_trade_performance from previous implementation
#the function is defined in the current environment
#Applied trade performance calculation function
#Printed all calculated metrics

     Step 4: Interpret the Results
After computing the metrics, interpreted the results focusing on the 10 chosen metrics. Each metric should provide insight into Nancy Pelosi's trading behavior and performance

In [77]:
import pandas as pd
import numpy as np
from datetime import datetime

# Example function to get ticker price (mock implementation)
def getTickerPrice(ticker: str, date: datetime) -> float:
    # Replace with actual implementation to fetch price for a given ticker and date
    return np.random.uniform(1, 100)

def calculate_trade_performance(trades_df):
    # Convert date columns to datetime format if needed
    trades_df['transactionDate'] = pd.to_datetime(trades_df['transactionDate'], errors='coerce')
    trades_df['disclosureDate'] = pd.to_datetime(trades_df['disclosureDate'], errors='coerce')

    # Convert amount and capitalGainsOver200USD to numeric if needed (assuming columns might be strings in some datasets)
    if trades_df['amount'].dtype == 'object':
        trades_df['amount'] = pd.to_numeric(trades_df['amount'].str.replace('[\$,]', '', regex=True), errors='coerce')
    if trades_df['capitalGainsOver200USD'].dtype == 'object':
        trades_df['capitalGainsOver200USD'] = pd.to_numeric(trades_df['capitalGainsOver200USD'].str.replace('[\$,]', '', regex=True), errors='coerce')

    # Initialize metrics dictionary
    metrics = {}

    if trades_df.empty:
        return metrics
    
    # 1. Total number of trades
    metrics['Total Trades'] = len(trades_df)

    # 2. Total buy and sell counts
    metrics['Buy Count'] = len(trades_df[trades_df['type'].str.contains('purchase', case=False)])
    metrics['Sell Count'] = len(trades_df[trades_df['type'].str.contains('sale', case=False)])

    # 3. Total capital gains
    metrics['Total Capital Gains'] = trades_df['capitalGainsOver200USD'].sum()

    # 4. Long strategy return (assuming sold higher than bought)
    long_returns = trades_df[trades_df['type'].str.contains('sale', case=False)]['capitalGainsOver200USD'].sum()
    metrics['Long Strategy Return'] = long_returns

    # 5. Short strategy return (assuming covering later at higher price)
    short_returns = trades_df[trades_df['type'].str.contains('purchase', case=False)]['capitalGainsOver200USD'].sum()
    metrics['Short Strategy Return'] = short_returns

    # 6. Average holding period
    trades_df['Holding Period'] = (trades_df['disclosureDate'] - trades_df['transactionDate']).dt.days
    metrics['Average Holding Period'] = trades_df['Holding Period'].mean()

    # 7. Win-Loss Ratio (handle division by zero)
    if short_returns != 0:
        metrics['Win-Loss Ratio'] = long_returns / short_returns
    else:
        metrics['Win-Loss Ratio'] = 0  # Set to 0 or handle appropriately

    # Placeholder value for Sharpe Ratio (requires risk-free rate and benchmark returns)
    metrics['Sharpe Ratio'] = 0.5  # Placeholder value

    # Calculate Max Drawdown (placeholder calculation)
    sale_amounts = trades_df.loc[trades_df['type'].str.contains('sale', case=False), 'amount'].values
    purchase_amounts = trades_df.loc[trades_df['type'].str.contains('purchase', case=False), 'amount'].values

    cumulative_return_sale = np.cumsum(sale_amounts)
    cumulative_return_purchase = np.cumsum(purchase_amounts)

    if len(cumulative_return_sale) > len(cumulative_return_purchase):
        cumulative_return_sale = cumulative_return_sale[:len(cumulative_return_purchase)]
    elif len(cumulative_return_purchase) > len(cumulative_return_sale):
        cumulative_return_purchase = cumulative_return_purchase[:len(cumulative_return_sale)]

    max_drawdown = np.max(np.maximum.accumulate(cumulative_return_purchase) - cumulative_return_sale) / np.max(np.maximum.accumulate(cumulative_return_purchase))
    metrics['Max Drawdown'] = max_drawdown

    return metrics

# Load Nancy Pelosi's trades data
file_path = 'testData.csv'
trades_df = pd.read_csv(file_path)

# Ensure column names match expected format by calculate_trade_performance function
trades_df = trades_df.rename(columns={
    'Date': 'transactionDate',
    'Symbol': 'ticker',
    'Side': 'type',
    'Size': 'amount',
    'Price': 'capitalGainsOver200USD',
    'DisclosureDate': 'disclosureDate'
})

# Convert 'type' to lowercase to match expected format ('buy' or 'sell')
trades_df['type'] = trades_df['type'].str.lower()

# Apply trade performance calculation function
performance_metrics = calculate_trade_performance(trades_df)

# Print all calculated metrics
print("\nTrade Performance Metrics:")
for key, value in performance_metrics.items():
    print(f"{key}: {value}")



Trade Performance Metrics:
Total Trades: 102
Buy Count: 40
Sell Count: 62
Total Capital Gains: 0.0
Long Strategy Return: 0.0
Short Strategy Return: 0.0
Average Holding Period: 39.53921568627451
Win-Loss Ratio: 0
Sharpe Ratio: 0.5
Max Drawdown: nan
